In [11]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [12]:
#import necessary modules for package

import wikipediaapi
import pandas as pd
import networkx as nx
import copy
from datetime import date
from itertools import zip_longest

In [13]:
#taking the more straightforward route, user supplies program with 2 wikipedia links themselves

search_node_a = input("enter first wikipedia page name: ")
search_node_b = input("enter second wikipedia page name: ")

enter first wikipedia page name: Kevin Bacon
enter second wikipedia page name: Jennifer Lawrence


In [14]:
#get Wikipedia page for the search nodes a, b

wiki_wiki = wikipediaapi.Wikipedia('en')

page_a = wiki_wiki.page(search_node_a)
page_b = wiki_wiki.page(search_node_b)

In [15]:
#f'ns used to get links from Wikipedia pages and clean page links list to remove Wikipedia attributes 

def return_links(page, list_n):
    list_n.clear()
    links = page.links
    for title in sorted(links.keys()):
        list_n.append(title)
            
def clean_links(list_node):
    for item in list_node.copy():
        if ((item in ('Wayback Machine', 'Specials (Unicode block)')) or (item.endswith(('(disambiguation)', '(identifier)'))) or (item.startswith(('List of ', 'Category:', 'File:', 'Help:', 'Talk:', 'Template:', 'Wikipedia:', 'Template talk:', 'Portal:', 'Wiki')))):
            list_node.remove(item)

In [16]:
list_node_a = []
list_node_b = []

return_links(page_a, list_node_a)
return_links(page_b, list_node_b)

clean_links(list_node_a)
clean_links(list_node_b)

In [17]:
#add page title to graph dict as key and nodes as values
#form the network from the graph dict 

graph = {}

graph[(page_a.title)] = list_node_a
graph[(page_b.title)] = list_node_b

G = nx.to_networkx_graph(graph)

In [18]:
page_links = []

error = ""
shortest_path_list = ""

combined_a_b = list_node_a + list_node_b

In [19]:
titles = combined_a_b

for t in titles:
    page = wiki_wiki.page(t)
    return_links(page, page_links)
    clean_links(page_links)
    
    graph[(page.title)] = copy.deepcopy(page_links)
    
    for p in page_links:
        if p not in titles:
            titles.append(p)
        else:
            break;
        break;
            
    page_links.clear();
    del page; 
    
    try:
        G = nx.to_networkx_graph(graph)
        nx.shortest_path(G, page_a.title, page_b.title)
    except nx.NetworkXNoPath:
        if t == titles[-1]: 
            error = "No path found between %s and %s" % (page_a.title, page_b.title)
            break;
        else:
            print(t)
            continue;
    else:
        shortest_path_list = nx.shortest_path(G, page_a.title, page_b.title)
        break;

    if error != "":
        break;

if error == "":
    print("There are " + str(len(shortest_path_list) - 1) + " degrees of separation between " + page_a.title + " and " + page_b.title + "\n")
    print(shortest_path_list)
else:
    print(error)

There are 2 degrees of separation between Kevin Bacon and Jennifer Lawrence

['Kevin Bacon', 'Academy Award', 'Jennifer Lawrence']


In [20]:
graph

{'Kevin Bacon': ['8 (play)',
  'A Few Good Men',
  'A Few Good Men (film)',
  'Academy Award',
  'Access Hollywood',
  'Al Pacino',
  'Alan Rickman',
  'Albert Finney',
  'Alexander Skarsgård',
  'AllMovie',
  'American Foundation for Equal Rights',
  'Andrew Lincoln',
  'Animal House',
  'Anthony Andrews',
  'Antireligion',
  'Apollo 13 (film)',
  'Atheism',
  'Atom Egoyan',
  'Balto (film)',
  'Barack Obama',
  'Barry Levinson',
  'Beau Bridges',
  'Beauty Shop',
  'Ben Browder',
  'Ben Kingsley',
  'Bernie Madoff',
  'Big screen',
  'Bill Nighy',
  'Billy Bob Thornton',
  'Black Mass (film)',
  'Blockbuster (entertainment)',
  'Blockbuster Entertainment Awards',
  'Bob Odenkirk',
  'Boston Society of Film Critics',
  'Boston Society of Film Critics Award for Best Cast',
  'Bravo Otto',
  'Brian Dennehy',
  'Broadcast Film Critics Association',
  'Bruce Campbell',
  'Bryan Cranston',
  'Bucknell University',
  'CableACE Award',
  'Cannes Film Festival',
  'Casey Affleck',
  'Chadwick

In [19]:
df = pd.DataFrame(
    [(k, i) for k, v in graph.items() for i in v], 
    columns=['key_id', 'ids']
)

df
#print((title))
#print(len(page_list))
#print(df["key_id"].unique().tolist())

,key_id,ids
0,War of 1812,13th United States Congress
1,War of 1812,1812 Baltimore riots
2,War of 1812,1828 United States presidential election
3,War of 1812,1838 Mormon War
4,War of 1812,1936–1939 Arab revolt in Palestine
...,...,...
3831,1917 South American Championship,Uruguay
3832,1917 South American Championship,Uruguay national football team
3833,1917 South American Championship,Uruguayan Football Association
3834,1917 South American Championship,Álvaro Saralegui


In [20]:
G2 = nx.to_networkx_graph(graph)
nx.shortest_path(G2, page_a.title, page_b.title)

['War of 1812', 'Chile', '1917 South American Championship', 'Lionel Messi']

In [21]:
df.to_excel("test9.xlsx", index = False)